In [1]:
import requests
import json
import pprint
import pandas as pd
import pycurl
from itertools import chain
import numpy as np

pp = pprint.PrettyPrinter()

In [2]:
# This is an example of one pull from the balldontlie.io API for a player searched by name
# You can see that lebron james was added as a search term at the end, and we can replicate this with other players
# The goal being to isolate the id's to use later for performance statistics
# Next we will be creating a for loop to get all of the players from our salaries list

In [3]:
# As you can see in this example, any capitalization does not matter for the player search
james = requests.get("https://www.balldontlie.io/api/v1/players/?search=LEbron jaMes")
james.json()

{'data': [{'id': 237,
   'first_name': 'LeBron',
   'height_feet': 6,
   'height_inches': 8,
   'last_name': 'James',
   'position': 'F',
   'team': {'id': 14,
    'abbreviation': 'LAL',
    'city': 'Los Angeles',
    'conference': 'West',
    'division': 'Pacific',
    'full_name': 'Los Angeles Lakers',
    'name': 'Lakers'},
   'weight_pounds': 250}],
 'meta': {'total_pages': 1,
  'current_page': 1,
  'next_page': None,
  'per_page': 25,
  'total_count': 1}}

In [4]:
# We take the file from BDLio and convert it to a dictionary so we car parse the ID
james_dict = json.loads(james.text)
print(james_dict)
type(james_dict)

{'data': [{'id': 237, 'first_name': 'LeBron', 'height_feet': 6, 'height_inches': 8, 'last_name': 'James', 'position': 'F', 'team': {'id': 14, 'abbreviation': 'LAL', 'city': 'Los Angeles', 'conference': 'West', 'division': 'Pacific', 'full_name': 'Los Angeles Lakers', 'name': 'Lakers'}, 'weight_pounds': 250}], 'meta': {'total_pages': 1, 'current_page': 1, 'next_page': None, 'per_page': 25, 'total_count': 1}}


dict

In [5]:
james_id = james_dict['data'][0]["id"]

In [6]:
%store -r player_names_list
player_names_list

['Stephen Curry',
 'John Wall',
 'Russell Westbrook',
 'James Harden',
 'Damian Lillard',
 'LeBron James',
 'Kevin Durant',
 'Giannis Antetokounmpo',
 'Paul George',
 'Kawhi Leonard',
 'Klay Thompson',
 'Jimmy Butler',
 'Tobias Harris',
 'Khris Middleton',
 'Anthony Davis',
 'Rudy Gobert',
 'Kyrie Irving',
 'Bradley Beal',
 'Kristaps Porzingis',
 'Devin Booker']

In [7]:
url = "https://www.balldontlie.io/api/v1/players/?search="
player_id_list = []

for name in player_names_list:
    player_url = url + f"{name}"
    player_get = requests.get(player_url)
    player_get.json()
    player_dict = json.loads(player_get.text)
    player_id_list.append(player_dict['data'][0]["id"])
print(player_id_list)


[115, 1014, 472, 192, 278, 237, 140, 15, 172, 274, 443, 79, 200, 315, 117, 176, 228, 37, 378, 57]


In [8]:
averages_url = "https://www.balldontlie.io/api/v1/season_averages?player_ids[]="
averages_list = []
for ids in player_id_list:
    averages_new = averages_url + f"{str(ids)}"
    averages_get = requests.get(averages_new)
    averages_get.json()
    averages_dict = json.loads(averages_get.text)
    averages_list.append(averages_dict['data'])
    if ids == 57:
        break
averages_list
  

[[{'games_played': 12,
   'player_id': 115,
   'season': 2021,
   'min': '32:28',
   'fgm': 8.25,
   'fga': 19.0,
   'fg3m': 4.83,
   'fg3a': 12.42,
   'ftm': 4.83,
   'fta': 5.08,
   'oreb': 0.75,
   'dreb': 5.58,
   'reb': 6.33,
   'ast': 6.42,
   'stl': 1.67,
   'blk': 0.5,
   'turnover': 3.17,
   'pf': 1.42,
   'pts': 26.17,
   'fg_pct': 0.434,
   'fg3_pct': 0.389,
   'ft_pct': 0.951}],
 [],
 [{'games_played': 12,
   'player_id': 472,
   'season': 2021,
   'min': '36:18',
   'fgm': 7.58,
   'fga': 18.25,
   'fg3m': 1.17,
   'fg3a': 4.17,
   'ftm': 3.0,
   'fta': 4.58,
   'oreb': 1.58,
   'dreb': 7.5,
   'reb': 9.08,
   'ast': 8.92,
   'stl': 1.5,
   'blk': 0.25,
   'turnover': 5.33,
   'pf': 3.42,
   'pts': 19.33,
   'fg_pct': 0.416,
   'fg3_pct': 0.28,
   'ft_pct': 0.655}],
 [{'games_played': 12,
   'player_id': 192,
   'season': 2021,
   'min': '33:53',
   'fgm': 5.58,
   'fga': 13.67,
   'fg3m': 2.92,
   'fg3a': 7.42,
   'ftm': 4.08,
   'fta': 4.67,
   'oreb': 1.17,
   'dreb': 6

In [9]:
averages_df = pd.DataFrame(list(chain.from_iterable(averages_list)))
averages_df

,games_played,player_id,season,min,fgm,fga,fg3m,fg3a,ftm,fta,...,reb,ast,stl,blk,turnover,pf,pts,fg_pct,fg3_pct,ft_pct
0,12,115,2021,32:28,8.25,19.00,4.83,12.42,4.83,5.08,...,6.33,6.42,1.67,0.50,3.17,1.42,26.17,0.434,0.389,0.951
1,12,472,2021,36:18,7.58,18.25,1.17,4.17,3.00,4.58,...,9.08,8.92,1.50,0.25,5.33,3.42,19.33,0.416,0.280,0.655
2,12,192,2021,33:53,5.58,13.67,2.92,7.42,4.08,4.67,...,8.00,8.75,1.08,0.83,4.67,2.33,18.17,0.409,0.393,0.875
3,13,278,2021,33:34,7.31,18.62,2.62,9.31,2.69,3.15,...,4.15,7.69,0.62,0.08,2.46,1.62,19.92,0.393,0.281,0.854
4,6,237,2021,37:03,9.50,20.33,2.83,8.17,3.00,3.83,...,5.50,7.00,2.33,0.83,4.17,2.33,24.83,0.467,0.347,0.783
5,12,140,2021,34:27,11.17,19.08,1.92,4.75,5.25,6.33,...,8.42,5.00,0.58,0.67,3.50,1.33,29.50,0.585,0.404,0.829
6,12,15,2021,32:52,9.50,19.17,1.33,4.67,6.25,9.08,...,11.83,6.00,1.08,1.75,3.00,3.00,26.58,0.496,0.286,0.688
7,11,172,2021,31:50,9.00,19.64,2.91,8.00,3.36,3.91,...,7.55,4.91,2.27,0.45,4.18,2.73,24.27,0.458,0.364,0.860
8,11,79,2021,32:59,8.00,15.18,0.45,1.55,7.18,8.45,...,5.45,5.09,2.09,0.36,1.91,2.00,23.64,0.527,0.294,0.849
9,7,200,2021,33:34,8.29,16.00,1.29,4.43,1.86,2.00,...,8.71,4.57,0.14,0.43,1.86,1.43,19.71,0.518,0.290,0.929


In [10]:
%store -r salary_df


In [11]:
player_id_df = pd.DataFrame(player_id_list)
salary_id_join = player_id_df.join(salary_df)

In [12]:
salary_player_id_df = salary_id_join.rename(columns={0: 'player_id'})
salary_player_id_df

,player_id,Rk,Player,Tm,2021-22
0,115,1,Stephen Curry,GSW,$45780966.00
1,1014,2,John Wall,HOU,$44310840.00
2,472,3,Russell Westbrook,LAL,$44211146.00
3,192,4,James Harden,BRK,$43848000.00
4,278,5,Damian Lillard,POR,$43750000.00
5,237,6,LeBron James,LAL,$41180544.00
6,140,7,Kevin Durant,BRK,$40918900.00
7,15,8,Giannis Antetokounmpo,MIL,$39344970.00
8,172,9,Paul George,LAC,$39344970.00
9,274,10,Kawhi Leonard,LAC,$39344900.00


In [13]:
all_data_df = salary_player_id_df.merge(averages_df, on='player_id', how = 'outer')
all_data_df.set_index('Rk').fillna(0)

,player_id,Player,Tm,2021-22,games_played,season,min,fgm,fga,fg3m,...,reb,ast,stl,blk,turnover,pf,pts,fg_pct,fg3_pct,ft_pct
Rk,,,,,,,,,,,,,,,,,,,,,
1,115,Stephen Curry,GSW,$45780966.00,12.0,2021.0,32:28,8.25,19.00,4.83,...,6.33,6.42,1.67,0.50,3.17,1.42,26.17,0.434,0.389,0.951
2,1014,John Wall,HOU,$44310840.00,0.0,0.0,0,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000
3,472,Russell Westbrook,LAL,$44211146.00,12.0,2021.0,36:18,7.58,18.25,1.17,...,9.08,8.92,1.50,0.25,5.33,3.42,19.33,0.416,0.280,0.655
4,192,James Harden,BRK,$43848000.00,12.0,2021.0,33:53,5.58,13.67,2.92,...,8.00,8.75,1.08,0.83,4.67,2.33,18.17,0.409,0.393,0.875
5,278,Damian Lillard,POR,$43750000.00,13.0,2021.0,33:34,7.31,18.62,2.62,...,4.15,7.69,0.62,0.08,2.46,1.62,19.92,0.393,0.281,0.854
6,237,LeBron James,LAL,$41180544.00,6.0,2021.0,37:03,9.50,20.33,2.83,...,5.50,7.00,2.33,0.83,4.17,2.33,24.83,0.467,0.347,0.783
7,140,Kevin Durant,BRK,$40918900.00,12.0,2021.0,34:27,11.17,19.08,1.92,...,8.42,5.00,0.58,0.67,3.50,1.33,29.50,0.585,0.404,0.829
8,15,Giannis Antetokounmpo,MIL,$39344970.00,12.0,2021.0,32:52,9.50,19.17,1.33,...,11.83,6.00,1.08,1.75,3.00,3.00,26.58,0.496,0.286,0.688
9,172,Paul George,LAC,$39344970.00,11.0,2021.0,31:50,9.00,19.64,2.91,...,7.55,4.91,2.27,0.45,4.18,2.73,24.27,0.458,0.364,0.860


In [14]:
# Game Score = Points Scored + (0.4 x Field Goals) – (0.7 x Field Goal Attempts) – (0.4 x (Free Throw Attempts – Free Throws)) + 
# (0.7 x Offensive Rebounds) + (0.3 x Defensive Rebounds) + Steals + (0.7 x Assists) + (0.7 x Blocks) – (0.4 x Personal Fouls) – Turnovers

In [15]:
game_scores_list = []
for player in range (len(all_data_df)):
    a = all_data_df.loc[player, 'pts']
    b = all_data_df.loc[player, 'fga'] * 0.4
    c = all_data_df.loc[player,'fga'] * 0.7
    d = ((all_data_df.loc[player,'fta']) - (all_data_df.loc[player, 'ftm']) * 0.4)
    e = all_data_df.loc[player,'oreb'] * 0.7
    f = all_data_df.loc[player,'dreb'] * 0.3
    g = all_data_df.loc[player,'stl']
    h = all_data_df.loc[player,'ast'] * 0.7
    i = all_data_df.loc[player,'blk'] * 0.7
    j = all_data_df.loc[player,'pf'] * 0.4
    k = all_data_df.loc[player,'turnover']
    game_score_player = a + b - c - d + e + f + g + h + i - j - k
    game_scores_list.append([game_score_player])
game_scores_list

[[22.297000000000004],
 [nan],
 [15.052000000000005],
 [16.083],
 [16.583000000000002],
 [20.659999999999997],
 [22.789],
 [20.873999999999995],
 [18.968000000000004],
 [nan],
 [nan],
 [19.032999999999994],
 [17.930999999999997],
 [14.434000000000001],
 [20.163000000000004],
 [11.482000000000001],
 [nan],
 [16.9],
 [11.793],
 [15.469000000000001]]

In [16]:
game_score_df = pd.DataFrame(game_scores_list)
game_score_df.fillna(0)


,0
0,22.297
1,0.000
2,15.052
3,16.083
4,16.583
5,20.660
6,22.789
7,20.874
8,18.968
9,0.000


In [17]:
salaries_df = all_data_df.iloc[: , 4].str.replace('$', '')
salaries_df.astype(float)

/Users/tgmyers/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


0     45780966.0
1     44310840.0
2     44211146.0
3     43848000.0
4     43750000.0
5     41180544.0
6     40918900.0
7     39344970.0
8     39344970.0
9     39344900.0
10    37980720.0
11    36016200.0
12    36000000.0
13    35500000.0
14    35361360.0
15    35344828.0
16    34916200.0
17    34502130.0
18    31650600.0
19    31610000.0
Name: 2021-22, dtype: float64

In [18]:
salaries_gamescore_nan = pd.merge(salaries_df, game_score_df, right_index=True, left_index=True)
salaries_gamescore_df = salaries_gamescore_nan.fillna(0)
salaries_gamescore_df['2021-22'] = salaries_gamescore_df['2021-22'].astype(float)
salaries_gamescore_df.dtypes

2021-22    float64
0          float64
dtype: object

In [19]:
salaries_gamescore_df = salaries_gamescore_df.rename(columns = {0 : 'game_score'})
salaries_gamescore_df

,2021-22,game_score
0,45780966.0,22.297
1,44310840.0,0.000
2,44211146.0,15.052
3,43848000.0,16.083
4,43750000.0,16.583
5,41180544.0,20.660
6,40918900.0,22.789
7,39344970.0,20.874
8,39344970.0,18.968
9,39344900.0,0.000


In [25]:
pd.options.display.float_format = '{:,.2f}'.format
salaries_gamescore_df['Result'] = salaries_gamescore_df["2021-22"]/salaries_gamescore_df['game_score']
salaries_gamescore_df.replace([np.inf, -np.inf], np.nan)
salaries_gamescore_df

NameError: name 'df' is not defined